In [1]:
from scraper import Scraper
import pandas as pd
import httpx
from bs4 import BeautifulSoup
from random import random, randint
import time
import pandas as pd
from datetime import date

In [2]:
s = Scraper()

In [3]:
cities = ['amsterdam']
#sites = ['pararius','funda','kamernet']
#sites = ['kamernet']
sites = ['pararius']
postTypes = ['Buy','Rent']

In [4]:
s.run(cities,sites,postTypes,10000)

Running Site: pararius
    Running post types: Buy
        Running city: amsterdam
            Running property type: appartement
            Page: 1
            Page: 2
            Page: 3
            Page: 4
            Page: 5


KeyboardInterrupt: 

In [58]:
list(s.properties['url'].values)

['https://www.pararius.nl/appartement-te-koop/amsterdam/dc556e18/akersingel',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/0c02681d/brouwersgracht',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/a1c9cd9f/laurierstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/837286f1/bilderdijkpark',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/0fdfe24e/tolstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/aba1f253/schovenstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/912045de/de-kempenaerstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/46c85613/baffinstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/4d9060e6/bloemstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/c6886a2e/h-diesveldsingel',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/69b6f950/duke-ellingtonstraat',
 'https://www.pararius.nl/appartement-te-koop/amsterdam/75f68bd3/h-gerhardstraat',
 'https://www.parar

In [107]:
update_df = history.set_index('url')[['first_scrape_date', 'last_scrape_date']].join(
    s.properties.set_index('url')['scrape_date'], 
    how='outer',
    rsuffix=''
)

In [134]:
new_properties = s.properties[s.properties['url'].isin(update_df[update_df['first_scrape_date'].isna()].index)].copy()

In [136]:
new_properties['first_scrape_date'] = new_properties['scrape_date']
new_properties['last_scrape_date'] = new_properties['scrape_date']
new_properties = new_properties.drop(columns=['scrape_date'])

In [112]:
properties_to_update = update_df[((~update_df['first_scrape_date'].isna())&
                                  (~update_df['scrape_date'].isna()))]

In [121]:
history = history.set_index('url')
history.loc[history.index.isin(properties_to_update.index),'last_scrape_date'] = properties_to_update['scrape_date']
history = history.reset_index()

In [138]:
history = pd.concat([history, new_properties])

In [20]:
history.to_csv('data/history_20241207.csv', index=False)

In [8]:
history = pd.read_csv('data/history_20241207.csv')

In [15]:
postcode_coord = pd.read_csv('data/postcode_coordinates.csv')

In [24]:
history['latitude'] = history['post_code'].map(postcode_coord.set_index('post_code')['latitude'].to_dict())
history['longitude'] = history['post_code'].map(postcode_coord.set_index('post_code')['longitude'].to_dict())

In [32]:
history = history.rename(columns={'postType': 'post_type', 'post_code': 'postcode'})

In [52]:
history.to_csv('data/history_20241207.csv', index=False)

In [39]:
history = history[
    ['source', 'first_scrape_date', 'last_scrape_date', 'url', 'post_type',
    'city', 'location', 'title', 'postcode', 'price', 'price_type', 'surface',
    'surface_unit', 'rooms', 'furnished', 'latitude', 'longitude']
]

In [49]:
history = history[~history['price'].isna()]

In [50]:
history['price'] = history['price'].astype(int)

/tmp/ipykernel_301026/4043373950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['price'] = history['price'].astype(int)


In [53]:
history.to_csv('data/history_20241207_nohead.csv', index=False, header=False)